In [1]:
!git clone https://github.com/AliaksandrSiarohin/first-order-model
# using first-order-model for deepfake models from https://github.com/AliaksandrSiarohin/first-order-model

!pip install ffmpeg-python

Cloning into 'first-order-model'...
remote: Enumerating objects: 4, done.
remote: Counting objects: 100% (4/4), done.
remote: Compressing objects: 100% (4/4), done.
remote: Total 243 (delta 0), reused 1 (delta 0), pack-reused 239
Receiving objects: 100% (243/243), 71.46 MiB | 31.05 MiB/s, done.
Resolving deltas: 100% (118/118), done.


https://drive.google.com/drive/folders/1kZ1gCnpfU0BnpdU47pLM_TQ6RypDDqgw

copy 'vox-cpk.pth.tar' file to your drive

In [3]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [4]:
cd first-order-model

/content/first-order-model


In [5]:
# import some libraries

import imageio
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from skimage.transform import resize
from IPython.display import HTML

from demo import load_checkpoints
from demo import make_animation
from skimage import img_as_ubyte

import ffmpeg

In [6]:
# input image 
source_image = imageio.imread('/content/gdrive/My Drive/Colab Notebooks/Test/Data/image.jpg')  # your image dir

# input video
video_source = imageio.mimread('/content/gdrive/My Drive/Colab Notebooks/Test/Data/base_video.mp4')   # your video dir

In [7]:
def display(img, video, generated=None):
    fig = plt.figure(figsize=(8 + 4 * (generated is not None), 6))

    ims = []
    for i in range(len(video)):
        cols = [img]
        cols.append(video[i])
        if generated is not None:
            cols.append(generated[i])
        im = plt.imshow(np.concatenate(cols, axis=1), animated=True)
        plt.axis('off')
        ims.append([im])

    ani = animation.ArtistAnimation(fig, ims, interval=50, repeat_delay=1000)
    plt.close()
    return ani

In [8]:
# resize image and video
source_image = resize(source_image, (256, 256))[..., :3]
video_source = [resize(frame, (256, 256))[..., :3] for frame in video_source]

In [9]:
# preview
HTML(display(source_image, video_source).to_html5_video())

In [11]:
output_name = 'final'

In [12]:
generator, kp_detector = load_checkpoints(config_path='/content/first-order-model/config/vox-256.yaml', 
                            checkpoint_path='/content/gdrive/My Drive/Colab Notebooks/Test/Data/vox-cpk.pth.tar')   # model dir

# create video 
predictions = make_animation(source_image, video_source, generator, kp_detector, relative=True)
imageio.mimsave(f'/content/gdrive/My Drive/Colab Notebooks/Test/Data/{output_name}.mp4', [img_as_ubyte(frame) for frame in predictions])   # image save dir

/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:3000: UserWarning: The default behavior for interpolate/upsample with float scale_factor changed in 1.6.0 to align with other frameworks/libraries, and uses scale_factor directly, instead of relying on the computed output size. If you wish to keep the old behavior, please set recompute_scale_factor=True. See the documentation of nn.Upsample for details. 
  warnings.warn("The default behavior for interpolate/upsample with float scale_factor changed "
  0%|          | 0/836 [00:00<?, ?it/s]/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:3384: UserWarning: Default grid_sample and affine_grid behavior has changed to align_corners=False since 1.3.0. Please specify align_corners=True if the old behavior is desired. See the documentation of grid_sample for details.
  warnings.warn("Default grid_sample and affine_grid behavior has changed "
/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:3121: UserWarning

In [13]:
# preview
HTML(display(source_image, video_source, predictions).to_html5_video())

In [ ]:
# extra features
# 1. Speed up video
# 2. Mix sound and video
# 3. Combine Video

In [21]:
# Speed up video
# exsample 3.0 x speed up

!ffmpeg -i\
 '/content/gdrive/My Drive/Colab Notebooks/Test/Data/final.mp4' \        # input_dir
 -filter:v \
 "setpts=0.33333*PTS" \                                                  # speedup setting
 '/content/gdrive/My Drive/Colab Notebooks/Test/Data/final_speed_up.mp4' # output_dir

ffmpeg version 3.4.8-0ubuntu0.2 Copyright (c) 2000-2020 the FFmpeg developers
  built with gcc 7 (Ubuntu 7.5.0-3ubuntu1~18.04)
  configuration: --prefix=/usr --extra-version=0ubuntu0.2 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --enable-gpl --disable-stripping --enable-avresample --enable-avisynth --enable-gnutls --enable-ladspa --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librubberband --enable-librsvg --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvorbis --enable-libvpx --enable-libwavpack --enable-libwebp --enable-libx265 --enable-libxml2 --enable-libxvid --enable-lib

In [24]:
# Mix sound and video

input_video = ffmpeg.input('/content/gdrive/My Drive/Colab Notebooks/Test/Data/final_speed_up.mp4')
input_audio = ffmpeg.input('/content/gdrive/My Drive/Colab Notebooks/Test/Data/sound.mp3')

ffmpeg.concat(input_video, input_audio, v=1, a=1).output('/content/gdrive/My Drive/Colab Notebooks/Test/Data/video_out.mp4').run()

(None, None)

In [30]:
# 3. Combine Video

!ffmpeg -i\
 '/content/gdrive/My Drive/Colab Notebooks/Test/Data/video_out_1.mp4' -i\     # input 1
 '/content/gdrive/My Drive/Colab Notebooks/Test/Data/video_out_2.mp4' \       # input 2
 -filter_complex hstack \
 '/content/gdrive/My Drive/Colab Notebooks/Test/Data/video_out_combine.mp4'   # output dir video

ffmpeg version 3.4.8-0ubuntu0.2 Copyright (c) 2000-2020 the FFmpeg developers
  built with gcc 7 (Ubuntu 7.5.0-3ubuntu1~18.04)
  configuration: --prefix=/usr --extra-version=0ubuntu0.2 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --enable-gpl --disable-stripping --enable-avresample --enable-avisynth --enable-gnutls --enable-ladspa --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librubberband --enable-librsvg --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvorbis --enable-libvpx --enable-libwavpack --enable-libwebp --enable-libx265 --enable-libxml2 --enable-libxvid --enable-lib